# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [55]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
import json
from pprint import pprint

# Import API key
from api_keys import geoapify_key
# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [56]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,smithers,54.7804,-127.1743,4.18,93,100,2.06,CA,1682326460
1,1,grytviken,-54.2811,-36.5092,1.22,80,24,12.61,GS,1682326461
2,2,labrador city,52.9463,-66.9114,-5.05,100,0,0.00,CA,1682326462
3,3,edinburgh of the seven seas,-37.0676,-12.3116,14.39,60,73,7.54,SH,1682326463
4,4,dudinka,69.4058,86.1778,2.57,80,29,6.56,RU,1682326464


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
%%capture --no-display
map_plot_humidity = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    color= 'City',
    size = 'Humidity',
    scale = 1,
    alpha = 0.5
)


# Display the map
map_plot_humidity

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [42]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df["Cloudiness"] = city_data_df["Cloudiness"].astype("Int64")
narrow_df = city_data_df.loc[(city_data_df["Max Temp"] < 30) & (city_data_df["Max Temp"] > 25) & (city_data_df["Cloudiness"] < 15) & (city_data_df["Wind Speed"] < 15)]
narrow_df.dropna()

# Display sample data
narrow_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,atafu village,-8.5421,-172.5159,28.47,74,12,2.19,TK,1682326466
8,8,fale old settlement,-9.3852,-171.2468,28.45,73,9,1.44,TK,1682326468
17,17,saint-pierre,-21.3393,55.4781,29.76,70,0,4.12,RE,1682326477
79,79,namibe,-15.1961,12.1522,26.43,60,4,4.66,AO,1682326543
87,87,mhamid,29.8200,-5.7200,28.61,18,7,5.12,MA,1682326555


### Step 3: Create a new DataFrame called `hotel_df`.

In [44]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
0,smithers,CA,54.7804,-127.1743,93,
1,grytviken,GS,-54.2811,-36.5092,80,
2,labrador city,CA,52.9463,-66.9114,100,
3,edinburgh of the seven seas,SH,-37.0676,-12.3116,60,
4,dudinka,RU,69.4058,86.1778,80,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [52]:
# Set parameters to search for a hotel
radius = 10000
params = { 
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # Get the city's name
    city = row["City"]
    
    # Get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"
   
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

#Display sample data
hotel_df

Starting hotel search
smithers - nearest hotel: Sunshine Inn Hotel
grytviken - nearest hotel: No hotel found
labrador city - nearest hotel: Carol Inn
edinburgh of the seven seas - nearest hotel: No hotel found
dudinka - nearest hotel: Виктория
middelburg - nearest hotel: No hotel found
atafu village - nearest hotel: No hotel found
port-aux-francais - nearest hotel: Keravel
fale old settlement - nearest hotel: No hotel found
palikir - national government center - nearest hotel: Mangrove Bay Hotel
hawaiian paradise park - nearest hotel: No hotel found
buga - nearest hotel: Hotel Cristo Rey
rakops - nearest hotel: Rakops River Lodge
melekeok village - nearest hotel: No hotel found
salinopolis - nearest hotel: Hotel Salinópolis
puerto natales - nearest hotel: Dorotea Patagonia Hostel
adamstown - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
bethel - nearest hotel: Hampton Inn Danbury
holualoa - nearest hotel: Kona Hotel
carnarvon - nearest hotel: No hotel found
t

kalianget - nearest hotel: No hotel found
luanda - nearest hotel: Hotel Forum
sheboygan - nearest hotel: Baymont Inn Plymouth
whakatane - nearest hotel: Whakatane Hotel
remire-montjoly - nearest hotel: Complexe Belova
la'ie - nearest hotel: Courtyard by Marriott
nova vicosa - nearest hotel: Pousada Pontal da Barra
todos santos - nearest hotel: Hotel Casa Tota
arraial do cabo - nearest hotel: No hotel found
sevastopol - nearest hotel: Барабулька
bardai - nearest hotel: No hotel found
mirabad - nearest hotel: No hotel found
fisterra - nearest hotel: Hotel Temático do Banco Azul
brookings - nearest hotel: Quality Inn
puerto santa - nearest hotel: No hotel found
mount isa - nearest hotel: Ibis Styles
port elizabeth - nearest hotel: Waterford Hotel
barah - nearest hotel: No hotel found
daru - nearest hotel: Kuki
kerikeri - nearest hotel: Avalon Resort
ketchikan - nearest hotel: Cape Fox Lodge
kingisepp - nearest hotel: Визит
dalbandin - nearest hotel: SF Al-dawood Restaurant
zabol - nearest

palu - nearest hotel: Ujung Pandang Hotel
saudarkrokur - nearest hotel: Mikligarður
hacienda la calera - nearest hotel: Restaurant Hotel La Bomba
taltal - nearest hotel: Hostería Taltal
buta - nearest hotel: Hôtel Rubi
nizhnesortymskiy - nearest hotel: Нижнесортымский
timbuktu - nearest hotel: Hotel La Maison
urrao - nearest hotel: Hotel El Castillo
vikhorevka - nearest hotel: Сосновые родники
anna regina - nearest hotel: Jaigobin Hotel
argo - nearest hotel: Apollon Hotel
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)
las toscas - nearest hotel: No hotel found
sun city - nearest hotel: WoodSpring Suites
sao francisco de assis - nearest hotel: Hotel Aguiar
lucea - nearest hotel: Blue Paloma Bed And Breakfast
manpur - nearest hotel: No hotel found
batesville - nearest hotel: No hotel found
mount pearl - nearest hotel: Sandman Signature St. John's
ulsteinvik - nearest hotel: Quality Hotel Ulstein
nogliki - nearest hotel: Ноглики
sinop - nearest hotel: No hotel foun

codrington - nearest hotel: No hotel found
neon karlovasion - nearest hotel: Ξενοδοχείο Αιγαίον
zouerate - nearest hotel: فندق تيرس زمور
lumut - nearest hotel: The Orient Star Resort Lumut
gizo - nearest hotel: Gizo Hotel
kingaroy - nearest hotel: No hotel found
darwin - nearest hotel: Mantra Pantanas Darwin
san patricio - nearest hotel: No hotel found
fredrikstad - nearest hotel: Hotell Valhalla
fillmore - nearest hotel: The Jail House Inn Bed & Breakfast
leini - nearest hotel: Hotel Air Palace


,City,Country,Lat,Lng,Humidity,Hotel Name
0,smithers,CA,54.7804,-127.1743,93,Sunshine Inn Hotel
1,grytviken,GS,-54.2811,-36.5092,80,No hotel found
2,labrador city,CA,52.9463,-66.9114,100,Carol Inn
3,edinburgh of the seven seas,SH,-37.0676,-12.3116,60,No hotel found
4,dudinka,RU,69.4058,86.1778,80,Виктория
...,...,...,...,...,...,...
577,darwin,AU,-12.4611,130.8418,63,Mantra Pantanas Darwin
578,san patricio,US,28.0170,-97.5169,78,No hotel found
579,fredrikstad,NO,59.2181,10.9298,91,Hotell Valhalla
580,fillmore,US,43.6666,-92.1002,68,The Jail House Inn Bed & Breakfast


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display
map_plot_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    # by="Country",
    # hover_cols=["Hotel Name"],
    geo=True,
    tiles="EsriImagery",
    frame_width=800,
    frame_height=600,
    size="Humidity",
    scaling_factor=0.5,  # changed `scale` to `scaling_factor`
    color="City",
    title="City Humidity Heatmap"  # added closing parenthesis
)


# Display the map plot
map_plot_hotel